# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
file = r'C:\Users\nnoar\OneDrive\Desktop\python-api-challenge\WeatherPy\weather.csv'
weather_df = pd.read_csv(file)
weather_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,chokurdakh,70.6333,147.9167,0.73,87.0,100.0,5.88,RU,1.631121e+09
1,hilo,19.7297,-155.0900,24.23,75.0,90.0,3.09,US,1.631121e+09
2,willmar,45.1219,-95.0433,22.01,49.0,1.0,6.69,US,1.631121e+09
3,attawapiskat,69.4541,-133.0374,8.36,72.0,51.0,4.48,CA,1.631121e+09
4,umzimvubu,20.6000,-99.1333,20.78,49.0,93.0,1.76,MX,1.631121e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#Configure gmaps
gmaps.configure(api_key=g_key)

#Set our variables
humidity = weather_df['Humidity']
locations = weather_df[['Lat','Lng']]

fig = gmaps.figure()

#Create our heatmap layer
heatmap_layer = gmaps.heatmap_layer(locations,weights=humidity,dissipating=False,max_intensity=max(weather_df['Humidity']), point_radius=2)

#Add our layer to the figure
fig.add_layer(heatmap_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [28]:
#Create my ideal weather conditions
ideal_df = weather_df.loc[(weather_df['Max Temp'] < 20) & (weather_df['Max Temp'] > 10)]
ideal_df = ideal_df.loc[ideal_df['Cloudiness'] == 0]
ideal_df = ideal_df.loc[ideal_df['Wind Speed'] < 10]
ideal_df = ideal_df.dropna()
ideal_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
23,cape town,-31.5889,28.7844,13.20,68.0,0.0,0.85,ZA,1.631121e+09
108,leningradskiy,43.6125,122.2653,11.66,71.0,0.0,2.65,CN,1.631121e+09
112,aquidauana,-24.8667,113.6333,13.04,100.0,0.0,3.09,AU,1.631121e+09
118,centralia,47.0056,28.8575,17.24,58.0,0.0,1.03,MD,1.631121e+09
127,tongliao,-37.7167,-72.2333,15.92,40.0,0.0,2.50,CL,1.631121e+09
151,chifeng,-33.7333,-70.7500,15.02,28.0,0.0,1.46,CL,1.631121e+09
165,klaksvik,-34.7000,-71.0500,15.21,52.0,0.0,3.36,CL,1.631121e+09
174,sabang,36.8648,70.8342,19.02,27.0,0.0,1.06,AF,1.631121e+09
184,buraydah,-35.3333,-72.4167,14.64,52.0,0.0,5.95,CL,1.631121e+09
191,lolua,-45.8667,-67.5000,17.92,36.0,0.0,6.69,AR,1.631121e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [35]:
hotel_df = ideal_df

#Create a Hotel Name column
hotel_df["Hotel Name"] = ""

#Create a parameters dictionary
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

#Loop to find Hotels
for index, row in hotel_df.iterrows():
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    params['location'] = f'{lat},{lng}'
    
    hotel_data = requests.get(base_url, params=params).json()
    
    try:
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data['results'][0]['name']
        
    except: 
        print('Could not find a hotel in', row['City'])
        
    
    

Could not find a hotel in sabang
Could not find a hotel in avarua
Could not find a hotel in birao


In [37]:
hotel_df = hotel_df[hotel_df['Hotel Name'] != ""]
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
23,cape town,-31.5889,28.7844,13.20,68.0,0.0,0.85,ZA,1.631121e+09,Mayfair Hotel
108,leningradskiy,43.6125,122.2653,11.66,71.0,0.0,2.65,CN,1.631121e+09,Country Garden Phoenix Hotel Tongliao
112,aquidauana,-24.8667,113.6333,13.04,100.0,0.0,3.09,AU,1.631121e+09,Hospitality Carnarvon
118,centralia,47.0056,28.8575,17.24,58.0,0.0,1.03,MD,1.631121e+09,Nobil Luxury Boutique Hotel
127,tongliao,-37.7167,-72.2333,15.92,40.0,0.0,2.50,CL,1.631121e+09,Hotel Mulchén
151,chifeng,-33.7333,-70.7500,15.02,28.0,0.0,1.46,CL,1.631121e+09,Motel INSIDE
165,klaksvik,-34.7000,-71.0500,15.21,52.0,0.0,3.36,CL,1.631121e+09,Vitalia B & B
184,buraydah,-35.3333,-72.4167,14.64,52.0,0.0,5.95,CL,1.631121e+09,Bancoarena Hotel
191,lolua,-45.8667,-67.5000,17.92,36.0,0.0,6.69,AR,1.631121e+09,WAM Hotel Patagónico
243,portland,63.4427,-20.2734,11.55,76.0,0.0,5.66,IS,1.631121e+09,Hotel Vestmannaeyjar


In [38]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [39]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(marker_layer)


# Display figure
fig


Figure(layout=FigureLayout(height='420px'))